In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.preprocessing import LabelEncoder

In [3]:
columns = [
    'movie_id', 'title', 'release_date', 'movie_popularity', 
    'vote_average', 'vote_count', 'budget', 'revenue', 'ratings', 'genres',
    'studios', 'actor_popularity_mean', 'director_popularity_mean'
]

target = ['revenue']

In [4]:
# Importing
# Load the data
file_path = Path('../Tables/ML_Test_pgs_checked.csv')
df = pd.read_csv(file_path, index_col=0)[:-2]
# , index_col=0
# df = df.loc[:, columns].copy()

# # Drop the null columns where all values are null
# df = df.dropna(axis='columns', how='all')

# # Drop the null rows
# df = df.dropna()

# Encoded Age Ratings
ratings_enc = {
    'G': 1,
    'PG': 2,
    'PG-13': 3,
    'R': 4,
    'NC-17': 5,
    'NR':6
}

df['ratings_enc'] = df['ratings'].apply(lambda x: ratings_enc[x])

df['release_month'] = pd.DatetimeIndex(df['release_date']).month
df.head()

,title,release_date,movie_popularity,vote_average,vote_count,budget,revenue,ratings,actor_popularity_mean,director_popularity_mean,studios,genres,ratings_enc,release_month
movie_id,,,,,,,,,,,,,,
671,Harry Potter and the Philosopher's Stone,2001-11-16,268.472,7.9,21429,125000000,976475550,PG,2.993965,2.566,"[""''HeydayFilms''"", ""''Other''"", ""''WarnerBros...","[""''Adventure''"", ""''Fantasy''""]",2,11
557,Spider-Man,2002-05-01,480.954,7.2,14421,139000000,821708551,PG-13,2.387022,2.914,"[""''ColumbiaPictures''"", ""''MarvelEntertainmen...","[""''Action''"", ""''Fantasy''""]",3,5
672,Harry Potter and the Chamber of Secrets,2002-11-13,246.027,7.7,17294,100000000,876688482,PG,2.696712,2.566,"[""''HeydayFilms''"", ""''Other''"", ""''WarnerBros...","[""''Adventure''"", ""''Fantasy''""]",2,11
673,Harry Potter and the Prisoner of Azkaban,2004-05-31,225.882,8.0,17001,130000000,789804554,PG,3.523069,3.333,"[""''HeydayFilms''"", ""''Other''"", ""''WarnerBros...","[""''Adventure''"", ""''Fantasy''""]",2,5
674,Harry Potter and the Goblet of Fire,2005-11-16,244.428,7.8,16341,150000000,895921036,PG-13,3.234944,2.695,"[""''HeydayFilms''"", ""''Other''"", ""''WarnerBros...","[""''Adventure''"", ""''Family''"", ""''Fantasy''""]",3,11


In [5]:
def profitability_df(df):
    if (df['revenue'] >= df['budget']*2):
        return True
    else:
        return False

df['profitability'] = df.apply(profitability_df, axis = 1)
df

,title,release_date,movie_popularity,vote_average,vote_count,budget,revenue,ratings,actor_popularity_mean,director_popularity_mean,studios,genres,ratings_enc,release_month,profitability
movie_id,,,,,,,,,,,,,,,
671,Harry Potter and the Philosopher's Stone,2001-11-16,268.472,7.9,21429,125000000,976475550,PG,2.993965,2.566,"[""''HeydayFilms''"", ""''Other''"", ""''WarnerBros...","[""''Adventure''"", ""''Fantasy''""]",2,11,True
557,Spider-Man,2002-05-01,480.954,7.2,14421,139000000,821708551,PG-13,2.387022,2.914,"[""''ColumbiaPictures''"", ""''MarvelEntertainmen...","[""''Action''"", ""''Fantasy''""]",3,5,True
672,Harry Potter and the Chamber of Secrets,2002-11-13,246.027,7.7,17294,100000000,876688482,PG,2.696712,2.566,"[""''HeydayFilms''"", ""''Other''"", ""''WarnerBros...","[""''Adventure''"", ""''Fantasy''""]",2,11,True
673,Harry Potter and the Prisoner of Azkaban,2004-05-31,225.882,8.0,17001,130000000,789804554,PG,3.523069,3.333,"[""''HeydayFilms''"", ""''Other''"", ""''WarnerBros...","[""''Adventure''"", ""''Fantasy''""]",2,5,True
674,Harry Potter and the Goblet of Fire,2005-11-16,244.428,7.8,16341,150000000,895921036,PG-13,3.234944,2.695,"[""''HeydayFilms''"", ""''Other''"", ""''WarnerBros...","[""''Adventure''"", ""''Family''"", ""''Fantasy''""]",3,11,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
744275,After We Fell,2021-09-01,1710.038,7.2,956,14000000,19000000,R,4.204667,2.580,"[""''Other''""]","[""''Drama''"", ""''Romance''""]",4,9,False
438631,Dune,2021-09-15,1363.113,8.0,4210,165000000,367000000,PG-13,5.641282,7.870,"[""'Other'""]","[""''Adventure''"", ""''ScienceFiction''""]",3,9,True
370172,No Time to Die,2021-09-29,3366.389,7.6,2075,242000000,734000000,PG-13,4.164382,4.167,"[""''Metro-Goldwyn-Mayer''"", ""''Other''"", ""''Un...","[""''Action''"", ""''Adventure''"", ""''Thriller''""]",3,9,True


In [6]:
# for test in df['studios']:
#     for studio in eval(test):
#         (studio)
        
# studio_revised = []
# for test in df['studios']:
#     holder = []
#     for studio in eval(test):
#         word = studio
#         if word not in holder:
#             holder.append(word)
#     studio_revised.append(holder)
    
# df.drop(['studios'],axis=1,inplace=True)
# df['studios'] = studio_revised

# df

In [7]:
# creating instance of labelencoder
labelencoder = LabelEncoder()

df['studios_cat'] = labelencoder.fit_transform(df.studios.values)
df['genres_cat'] = labelencoder.fit_transform(df.genres.values)

In [8]:
df = df.drop(['release_date', 'title', 'ratings', 'revenue'], axis=1)
df = df.drop(['studios', 'genres'], axis=1)
df.head()

,movie_popularity,vote_average,vote_count,budget,actor_popularity_mean,director_popularity_mean,ratings_enc,release_month,profitability,studios_cat,genres_cat
movie_id,,,,,,,,,,,
671,268.472,7.9,21429,125000000,2.993965,2.566,2,11,True,6,25
557,480.954,7.2,14421,139000000,2.387022,2.914,3,5,True,1,15
672,246.027,7.7,17294,100000000,2.696712,2.566,2,11,True,6,25
673,225.882,8.0,17001,130000000,3.523069,3.333,2,5,True,6,25
674,244.428,7.8,16341,150000000,3.234944,2.695,3,11,True,6,23


In [9]:
# Create our features
X = pd.get_dummies(df.drop(columns='profitability'))


# Create our target
y = pd.get_dummies(df['profitability'])

In [10]:
X.head()

,movie_popularity,vote_average,vote_count,budget,actor_popularity_mean,director_popularity_mean,ratings_enc,release_month,studios_cat,genres_cat
movie_id,,,,,,,,,,
671,268.472,7.9,21429,125000000,2.993965,2.566,2,11,6,25
557,480.954,7.2,14421,139000000,2.387022,2.914,3,5,1,15
672,246.027,7.7,17294,100000000,2.696712,2.566,2,11,6,25
673,225.882,8.0,17001,130000000,3.523069,3.333,2,5,6,25
674,244.428,7.8,16341,150000000,3.234944,2.695,3,11,6,23


In [11]:
# Check the balance of our target values
y = df['profitability']
y.value_counts()

True     50
False    20
Name: profitability, dtype: int64

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
Counter(y_train)

Counter({False: 13, True: 39})

# Balanced Random Forest Classifier

In [13]:
# Resample the training data with the BalancedRandomForestClassifier
from sklearn.preprocessing import StandardScaler
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.datasets import make_classification

scaler = StandardScaler()
X_scaler = scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

brfc = BalancedRandomForestClassifier(n_estimators=100)
brfc.fit(X_train_scaled, y_train)

BalancedRandomForestClassifier()

In [14]:
# Calculated the balanced accuracy score
y_pred = brfc.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.5

In [15]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[ 0,  7],
       [ 0, 11]], dtype=int64)

In [16]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

      False       0.00      0.00      1.00      0.00      0.00      0.00         7
       True       0.61      1.00      0.00      0.76      0.00      0.00        11

avg / total       0.37      0.61      0.39      0.46      0.00      0.00        18



In [17]:
# List the features sorted in descending order by feature importance
sorted(zip(brfc.feature_importances_, X.columns), reverse=True)

[(0.18354324502328417, 'vote_count'),
 (0.1735029929209354, 'studios_cat'),
 (0.15288502708666687, 'director_popularity_mean'),
 (0.09622990883916088, 'budget'),
 (0.07785230486948237, 'vote_average'),
 (0.07312427630025346, 'genres_cat'),
 (0.06938816829873433, 'movie_popularity'),
 (0.06757650144439038, 'actor_popularity_mean'),
 (0.05580709236533446, 'release_month'),
 (0.05009048285175772, 'ratings_enc')]

# Easy Ensemble AdaBoost Classifier

In [18]:
# Train the EasyEnsembleClassifier
from imblearn.ensemble import EasyEnsembleClassifier

eec = EasyEnsembleClassifier(n_estimators=300, random_state=1)
eec.fit(X_train, y_train)

EasyEnsembleClassifier(n_estimators=300, random_state=1)

In [19]:
# Calculated the balanced accuracy score
y_pred = eec.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.8181818181818181

In [20]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[7, 0],
       [4, 7]], dtype=int64)

In [21]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

      False       0.64      1.00      0.64      0.78      0.80      0.66         7
       True       1.00      0.64      1.00      0.78      0.80      0.61        11

avg / total       0.86      0.78      0.86      0.78      0.80      0.63        18



# Extra ML

In [22]:
# Creating a StandardScaler instance.
scaler = StandardScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [23]:
# Creating the decision tree classifier instance.
model = tree.DecisionTreeClassifier()
# Fitting the model.
model = model.fit(X_train_scaled, y_train)

In [24]:
# Making predictions using the testing data.
predictions = model.predict(X_test_scaled)

predictions

array([ True,  True,  True, False,  True,  True, False, False, False,
       False,  True, False, False, False,  True,  True, False, False])